# 📋 Grader Mapping — Assignment Deliverables

| Rubric Item | Notebook & Cell |
|-------------|----------------|
| EDA Visualizations | **01_EDA** — Cells 3–8 |
| Sentence Embeddings (SBERT) | **02_Embeddings** — Cells 2–4 |
| FAISS Indexing & Semantic Search | **03_Search** — Cells 2–5 |
| RAG Pipeline | **04_RAG** — Cells 2–4 |
| Evaluation Metrics (P@K, R@K) | **05_Evaluation** — Cells 2–4 |
| Business Insights Report | **06_Benchmarking** final section + `docs/Business_Insights_Report.md` |
| Index Benchmarking | **06_Benchmarking** — Cells 2–4 |

# 📊 Notebook 01: Exploratory Data Analysis & Preprocessing

**Flipkart Product Reviews Dataset**

This notebook performs comprehensive EDA on the Flipkart product review dataset, including:
- Dataset overview and cleaning
- Rating and sentiment distribution
- Product-level analytics
- Text length analysis
- Word clouds per sentiment class
- Product × Sentiment heatmap

In [ ]:
# ── Setup ──────────────────────────────────────────────────────────
import sys, os
sys.path.insert(0, os.path.abspath('..'))
os.environ.setdefault('SAMPLE_ONLY', 'true')  # CI-safe default

from src.config import Config
from src.data_ingest import load_flipkart, get_product_names
from src.visualization import (
    plot_rating_distribution,
    plot_sentiment_distribution,
    plot_text_length_distribution,
    plot_product_rating_heatmap,
    plot_wordcloud,
)

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

cfg = Config()
print(f'Sample mode: {cfg.SAMPLE_ONLY}  |  Max rows: {cfg.n_rows}')

In [ ]:
# ── Load and inspect dataset ───────────────────────────────────────
df = load_flipkart(cfg)
print(f'Dataset shape: {df.shape}')
print(f'Columns: {list(df.columns)}')
print(f'Products: {get_product_names(df, cfg)}')
print(f'\nSample rows:')
df.head()

In [ ]:
# ── Rating Distribution ────────────────────────────────────────────
fig = plot_rating_distribution(df[cfg.COL_RATING].values)
plt.show()

In [ ]:
# ── Sentiment Distribution ────────────────────────────────────────
fig = plot_sentiment_distribution(df[cfg.COL_SENTIMENT].values)
plt.show()

print('\n⚠️ Key insight: Dataset is heavily imbalanced — ~81% positive, ~14% negative, ~5% neutral')

In [ ]:
# ── Text Length Analysis ───────────────────────────────────────────
fig = plot_text_length_distribution(df['combined_text'].values)
plt.show()

print(f'Average combined_text length: {df["combined_text"].str.len().mean():.1f} chars')
print(f'Average Review length: {df[cfg.COL_REVIEW].str.len().mean():.1f} chars')
print(f'Average Summary length: {df[cfg.COL_SUMMARY].str.len().mean():.1f} chars')
print('\n💡 Reviews are very short (~12 chars), so we combine Summary + Review for richer embeddings.')

In [ ]:
# ── Product × Sentiment Heatmap ───────────────────────────────────
fig = plot_product_rating_heatmap(df, cfg.COL_PRODUCT, cfg.COL_RATING, cfg.COL_SENTIMENT)
plt.show()

In [ ]:
# ── Word Clouds by Sentiment ──────────────────────────────────────
for sentiment in ['positive', 'negative', 'neutral']:
    subset = df[df[cfg.COL_SENTIMENT] == sentiment]['combined_text']
    if len(subset) > 10:
        fig = plot_wordcloud(subset.values, title=f'Word Cloud — {sentiment.title()} Reviews')
        if fig: plt.show()

In [ ]:
# ── Product-level statistics ───────────────────────────────────────
product_stats = df.groupby(cfg.COL_PRODUCT).agg(
    count=(cfg.COL_RATING, 'count'),
    avg_rating=(cfg.COL_RATING, 'mean'),
    pct_positive=(cfg.COL_SENTIMENT, lambda x: (x == 'positive').mean() * 100),
).round(2).sort_values('avg_rating', ascending=False)

product_stats.index = [n[:40] + '...' if len(n) > 40 else n for n in product_stats.index]
print('📊 Product-Level Statistics:')
product_stats

### 🔍 EDA Summary

**Key Findings:**
1. **9 unique products** across electronics, appliances, and accessories
2. **Strong positive skew** — 81% positive sentiment, only 5% neutral
3. **Very short reviews** (avg ~12 chars) — we combine with Summary for meaningful embeddings
4. **Rating ≈ Sentiment** — high correlation as expected, but not 1:1
5. **Product variation** — some products have significantly different satisfaction levels

These insights directly inform our search system design:
- Short texts → no chunking needed (embed full combined_text)
- 9 products → feasible to add product-aware filtering
- Sentiment imbalance → evaluation should be stratified